In [11]:
from os.path import join, basename, abspath, exists
from os import listdir, makedirs

from keras import backend as K
from keras.models import load_model

from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
import cv2
from statistics import mean 

import time
import pickle
# for scaling and inverse_transform
from sklearn.preprocessing import MinMaxScaler

from jaad_data import JAAD

In [12]:
def bb_intersection_over_union(boxA, boxB):
	# determine the (x, y)-coordinates of the intersection rectangle
	xA = max(boxA[0], boxB[0])
	yA = max(boxA[1], boxB[1])
	xB = min(boxA[2], boxB[2])
	yB = min(boxA[3], boxB[3])
 
	# compute the area of intersection rectangle
	interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)
 
	# compute the area of both the prediction and ground-truth
	# rectangles
	boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
	boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)
 
	# compute the intersection over union by taking the intersection
	# area and dividing it by the sum of prediction + ground-truth
	# areas - the interesection area
	iou = interArea / float(boxAArea + boxBArea - interArea)
 
	# return the intersection over union value
	return iou

In [13]:
#model_path = "30f_future_epoch-lstm-model.h5"
model_path = "30f_future-model.h5"

model = load_model(model_path)
n_seq = 30
n_seq_future = 30

In [14]:
jaad_path = "/home/kara9147/jaad2"
annotation_path = join(jaad_path, 'annotations')
imdb = JAAD(data_path = jaad_path)

In [15]:
#videos = ["video_0%s"%str(i) for i in range(317,347)]
videos = ["video_00%s"%str(i) for i in range(71,100)]

In [16]:
len(videos)

29

In [17]:
avg_prediction_speed = list()
accuracy = list()
#pickle_in = open("min-max-scaler.pkl","rb")
#scaler = pickle.load(pickle_in)

In [18]:
for v in videos:
    anno = imdb._get_annotations(vid)

    bb_cross={}
    ped_b = [k for k,v in anno["ped_annotations"].items() if k.endswith('b')]

    for i, p in enumerate(ped_b):
        #if (1 in anno["ped_annotations"][p]["behavior"]["cross"]):
            bb_cross[p] = (anno["ped_annotations"][p]["bbox"])
            print(p)
    print(ped_b)
    
    for p in ped_b:
      #  print(v, p)
        lstm_seq = ( bb_cross[p])
        print(type(lstm_seq))
        frames = (anno['ped_annotations'][p]['frames'][0],
        len(anno['ped_annotations'][p]['frames']))

        df = pd.DataFrame(lstm_seq)
        df = df.astype('float32')

        #print(type(scaler))
        #print(scaler.data_max_)
        #print(scaler.data_min_)
        scaler = MinMaxScaler(feature_range=(0, 1))
        lstm_seq = scaler.fit_transform(lstm_seq)
        pred = True
        i = 0
        total_time = 0.0
        total_positive = 0

        while( i < lstm_seq.shape[0] - (n_seq + n_seq_future) -1):
            batch = lstm_seq[i:i + n_seq] # n_seq number of element with last index (n_seq-1)

            current = time.time()
            ##################################PREDICTION######################
            y_hat = model.predict(np.expand_dims(batch, axis=0))
            #print(y_hat)
            #print(len(batch))

            end = time.time()
            diff = end - current
            total_time  = total_time  + diff
            #print(end - current)
            #print("Time spent for predicting: {0}".format(diff))

            # Decode the raw prediction `y_pred`
            inv_yhat = scaler.inverse_transform(y_hat)[0]
            #print(inv_yhat)

            # future bounding box from gt

            boxA =  df[i+(n_seq + n_seq_future) - 1 :i + (n_seq + n_seq_future)].values[0]
            # future bounding box from prediction
            boxB = inv_yhat

            #print(boxA)
            #print(boxB)

            score = bb_intersection_over_union(boxA, boxB)
            if score > 0.5 :
                total_positive += 1

            # for the next bb
            i = i + 1

        print ("total #prediction: %d" % i)
        print ("total time: %f" % total_time)
        print ("avg prediction speed: %f" % (total_time / i))
        print ("total positive: %d" % total_positive)
        print ("accuracy: %f" % (total_positive / i))

        avg_prediction_speed.append(total_time / i) 
        accuracy.append(total_positive / i)
print(mean(avg_prediction_speed))
print(mean(accuracy))

0_317_2534b
0_317_2529b
0_317_2533b
0_317_2530b
['0_317_2534b', '0_317_2529b', '0_317_2533b', '0_317_2530b']
<class 'list'>
total #prediction: 189
total time: 3.211471
avg prediction speed: 0.016992
total positive: 34
accuracy: 0.179894
<class 'list'>
total #prediction: 143
total time: 2.300908
avg prediction speed: 0.016090
total positive: 70
accuracy: 0.489510
<class 'list'>
total #prediction: 98
total time: 1.595435
avg prediction speed: 0.016280
total positive: 15
accuracy: 0.153061
<class 'list'>
total #prediction: 195
total time: 3.159271
avg prediction speed: 0.016201
total positive: 68
accuracy: 0.348718
0_317_2534b
0_317_2529b
0_317_2533b
0_317_2530b
['0_317_2534b', '0_317_2529b', '0_317_2533b', '0_317_2530b']
<class 'list'>
total #prediction: 189
total time: 3.025024
avg prediction speed: 0.016005
total positive: 34
accuracy: 0.179894
<class 'list'>
total #prediction: 143
total time: 2.329544
avg prediction speed: 0.016291
total positive: 70
accuracy: 0.489510
<class 'list'>


total #prediction: 143
total time: 2.308755
avg prediction speed: 0.016145
total positive: 70
accuracy: 0.489510
<class 'list'>
total #prediction: 98
total time: 1.579133
avg prediction speed: 0.016114
total positive: 15
accuracy: 0.153061
<class 'list'>
total #prediction: 195
total time: 3.131235
avg prediction speed: 0.016058
total positive: 68
accuracy: 0.348718
0_317_2534b
0_317_2529b
0_317_2533b
0_317_2530b
['0_317_2534b', '0_317_2529b', '0_317_2533b', '0_317_2530b']
<class 'list'>
total #prediction: 189
total time: 3.077285
avg prediction speed: 0.016282
total positive: 34
accuracy: 0.179894
<class 'list'>
total #prediction: 143
total time: 2.302139
avg prediction speed: 0.016099
total positive: 70
accuracy: 0.489510
<class 'list'>
total #prediction: 98
total time: 1.597627
avg prediction speed: 0.016302
total positive: 15
accuracy: 0.153061
<class 'list'>
total #prediction: 195
total time: 3.150333
avg prediction speed: 0.016156
total positive: 68
accuracy: 0.348718
0_317_2534b


total #prediction: 195
total time: 3.070408
avg prediction speed: 0.015746
total positive: 68
accuracy: 0.348718
0_317_2534b
0_317_2529b
0_317_2533b
0_317_2530b
['0_317_2534b', '0_317_2529b', '0_317_2533b', '0_317_2530b']
<class 'list'>
total #prediction: 189
total time: 2.995238
avg prediction speed: 0.015848
total positive: 34
accuracy: 0.179894
<class 'list'>
total #prediction: 143
total time: 2.268537
avg prediction speed: 0.015864
total positive: 70
accuracy: 0.489510
<class 'list'>
total #prediction: 98
total time: 1.556436
avg prediction speed: 0.015882
total positive: 15
accuracy: 0.153061
<class 'list'>
total #prediction: 195
total time: 3.086299
avg prediction speed: 0.015827
total positive: 68
accuracy: 0.348718
0_317_2534b
0_317_2529b
0_317_2533b
0_317_2530b
['0_317_2534b', '0_317_2529b', '0_317_2533b', '0_317_2530b']
<class 'list'>
total #prediction: 189
total time: 3.005013
avg prediction speed: 0.015900
total positive: 34
accuracy: 0.179894
<class 'list'>
total #predicti

In [16]:
#i = 0

#while( i < lstm_seq.shape[0] - (n_seq + n_seq_future)  - 1):
#    print(i)
#    print(lstm_seq[i+59:i+60])
#    print(reframed.iloc[i:i+1, n_seq*4:n_seq*4 + 4]) 
#    i = i + 1
